In [ ]:
!pip install langchain_community

In [ ]:
!pip install pyDecision

In [ ]:

"""
A simplified MCDM fuzzy Best–Worst Method (BWM) analysis using one expert's input.

This script uses LangChain’s conversation chain to simulate an expert evaluation
and then applies the fuzzy BWM (from the pyDecision package) to compute weights.

Dependencies (install via requirements.txt or pip):
  - langchain_community
  - langchain_openai
  - pyDecision
  - numpy
  - pandas
"""

import os
import re
import numpy as np
import pandas as pd

from pyDecision.algorithm import fuzzy_bw_method
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory

# Get your OpenAI API key from an environment variable (or replace with your key)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "replace here the key")


def setup_conversation_chain(api_key=OPENAI_API_KEY, model="gpt-4", memory_window=1):
    """
    Sets up and returns a conversation chain using LangChain.

    :param api_key: Your OpenAI API key.
    :param model: The language model to use (e.g., 'gpt-4').
    :param memory_window: Number of past messages to store in memory.
    :return: A ConversationChain instance.
    """
    llm = ChatOpenAI(
        temperature=0.8,
        model=model,
        openai_api_key=api_key,
        verbose=False
    )
    conversation = ConversationChain(
        llm=llm,
        memory=ConversationBufferWindowMemory(k=memory_window),
        verbose=False
    )
    return conversation
    conversation.memory.clear()


def get_expert_response(conversation, prompt):
    """
    Sends a prompt to the conversation chain and returns the expert response.

    :param conversation: The ConversationChain instance.
    :param prompt: A string containing the prompt for the expert.
    :return: The LLM's response as a string.
    """
    response = conversation.predict(input=prompt)
    return response


def perform_fuzzy_bwm(mic, lic, eps_penalty=0):
    """
    Performs fuzzy BWM analysis given the 'most important' (mic) and 'least important' (lic)
    fuzzy comparisons.

    :param mic: List of fuzzy comparison tuples for the best criterion.
    :param lic: List of fuzzy comparison tuples for the worst criterion.
    :param eps_penalty: Penalty parameter for the BWM algorithm.
    :return: Tuple (epsilon, consistency ratio, fuzzy weights, crisp weights)
    """
    epsilon, cr, f_weights, weights = fuzzy_bw_method(mic, lic, eps_penalty=eps_penalty, verbose=True)
    return epsilon, cr, f_weights, weights


def parse_final_weights(response_text):
    """
    Parses a text response to extract final criteria weights along with the consistency ratio
    and epsilon value.

    :param response_text: The text containing the weight information.
    :return: A pandas DataFrame with columns: Criteria, Final Weight (%), CR, Epsilon Value.
    """
    consistency_info = re.search(r"Epsilon Value:\s*([\d\.]+)\s*CR:\s*([\d\.]+)", response_text)
    if consistency_info:
        epsilon_value = float(consistency_info.group(1))
        cr_value = float(consistency_info.group(2))
    else:
        epsilon_value = cr_value = None

    # Look for lines like "Integration (C1): 17.79%"
    final_criteria = re.findall(r"([A-Za-z\s\(\)0-9]+):\s*([\d\.]+)%", response_text)
    df_final = pd.DataFrame(final_criteria, columns=["Criteria", "Final Weight (%)"])
    if not df_final.empty:
        df_final["Final Weight (%)"] = df_final["Final Weight (%)"].astype(float)
        df_final["CR"] = cr_value
        df_final["Epsilon Value"] = epsilon_value
    return df_final


def main():
    # -------------------------
    # Setup: Create the conversation chain.
    # -------------------------
    conversation = setup_conversation_chain()

    # -------------------------
    # Expert Analysis – MCDM Expert
    # -------------------------
    prompt = """
You are an expert in multi-criteria decision-making (MCDM).
Your task is to analyze the following six criteria based on:
  - Relevance to overall project success
  - Impact on team collaboration
  - Effectiveness in resource management
  - Importance in meeting deadlines

The criteria are:
  1. Integration
  2. Communication
  3. Project Administration
  4. Human Resources
  5. Technology Factors
  6. Time

Provide a structured response with:
  1. The **best** criterion and why it is most important.
  2. The **worst** criterion and why it is least important.
    """
    expert_response = get_expert_response(conversation, prompt)
    print("=== Expert Response ===\n")
    print(expert_response)

    # -------------------------
    # Fuzzy BWM Analysis
    # -------------------------
    # For this example, we assume that the expert identifies "Integration" as the best criterion
    # and "Time" as the worst criterion. (Adjust these comparisons based on the expert's response if needed.)
    mic = [
        (1, 1, 1),       # Integration vs Integration
        (2/3, 1, 3/2),   # Integration vs Communication
        (2/3, 1, 3/2),   # Integration vs Project Administration
        (3/2, 2, 5/2),   # Integration vs Human Resources
        (5/2, 3, 7/2),   # Integration vs Technology Factors
        (7/2, 4, 9/2)    # Integration vs Time
    ]
    lic = [
        (2/3, 1, 3/2),   # Time vs Integration
        (3/2, 2, 5/2),   # Time vs Communication
        (3/2, 2, 5/2),   # Time vs Project Administration
        (5/2, 3, 7/2),   # Time vs Human Resources
        (7/2, 4, 9/2),   # Time vs Technology Factors
        (1, 1, 1)        # Time vs Time
    ]
    print("\n--- Fuzzy BWM Analysis ---")
    epsilon, cr, f_weights, weights = perform_fuzzy_bwm(mic, lic)
    criteria_names = [
        "Integration", "Communication", "Project Administration",
        "Human Resources", "Technology Factors", "Time"
    ]
    print("\nCrisp Weights:")
    for i, w in enumerate(weights):
        print(f"  {criteria_names[i]}: {round(w * 100, 2)}%")


if __name__ == "__main__":
    main()


In [ ]:

"""
Fuzzy BWM Analysis for Human Response

This script performs a fuzzy Best–Worst Method (BWM) analysis using the following setup:
  - The most important criterion (MIC) is Human Resources (C4).
  - The least important criterion (LIC) is Project Administration (C3).

Linguistic Terms and Membership Functions:
  EI: (1, 1, 1)
  WI: (2/3, 1, 3/2)
  FI: (3/2, 2, 5/2)
  VI: (5/2, 3, 7/2)
  AI: (7/2, 4, 9/2)

The criteria under consideration are:
  C1: Integration
  C2: Communication
  C3: Project Administration
  C4: Human Resources
  C5: Technology Factors
  C6: Time
"""

from pyDecision.algorithm import fuzzy_bw_method

# Define the Most Important Criteria (mic) comparisons.
# Here, the expert has selected Human Resources (C4) as the MIC.
# Each tuple compares a given criterion (C1, C2, C3, C4, C5, C6) against C4.
mic = [
    (3/2, 2, 5/2),   # C1 (Integration) vs C4 (Human Resources)
    (5/2, 3, 7/2),   # C2 (Communication) vs C4 (Human Resources)
    (2/3, 1, 3/2),   # C3 (Project Administration) vs C4 (Human Resources)
    (1, 1, 1),       # C4 (Human Resources) vs C4 (Human Resources)
    (3/2, 2, 5/2),   # C5 (Technology Factors) vs C4 (Human Resources)
    (5/2, 3, 7/2)    # C6 (Time) vs C4 (Human Resources)
]

# Define the Least Important Criteria (lic) comparisons.
# Here, the expert has selected Project Administration (C3) as the LIC.
# Each tuple compares a given criterion (C1, C2, C3, C4, C5, C6) against C3.
lic = [
    (2/3, 1, 3/2),   # C1 (Integration) vs C3 (Project Administration)
    (3/2, 2, 5/2),   # C2 (Communication) vs C3 (Project Administration)
    (1, 1, 1),       # C3 (Project Administration) vs C3 (Project Administration)
    (2/3, 1, 3/2),   # C4 (Human Resources) vs C3 (Project Administration)
    (3/2, 2, 5/2),   # C5 (Technology Factors) vs C3 (Project Administration)
    (2/3, 1, 3/2)    # C6 (Time) vs C3 (Project Administration)
]

# Call the fuzzy BWM function.
# eps_penalty = 0 indicates a more tolerant approach regarding epsilon minimization.
epsilon, cr, f_weights, weights = fuzzy_bw_method(mic, lic, eps_penalty=0, verbose=True)

# Print the fuzzy weights.
print("Fuzzy Weights:")
for i in range(len(f_weights)):
    print(f'fw(g{i+1}): {f_weights[i]}')

# Print the crisp weights (rounded for clarity).
print("\nCrisp Weights:")
for i in range(len(weights)):
    print(f'w(g{i+1}): {round(weights[i], 8)}')

# Define the names of the criteria.
criteria_names = [
    "Integration (C1)",
    "Communication (C2)",
    "Project Administration (C3)",
    "Human Resources (C4)",
    "Technology Factors (C5)",
    "Time (C6)"
]

# Print the weights in percentage form with their criterion names.
print("\nCriteria Weights (in %):")
for i in range(len(weights)):
    print(f'{criteria_names[i]}: {round(weights[i] * 100, 2)}%')
